## Pre Process


## Post Process


#### init function

In [2]:
import os, sys, random
from lib.library import delete_similar_image_in_subfolders, add_folder_name_to_files, rename_subfolders

[06/16/24 15:33:26] WARNING  Your inference package version 0.11.2 is out of date! Please upgrade to ]8;id=810379;file://d:\Project\Code\Diffusion_Data_Tools\myenv\lib\site-packages\inference\core\__init__.py\__init__.py]8;;\:]8;id=222130;file://d:\Project\Code\Diffusion_Data_Tools\myenv\lib\site-packages\inference\core\__init__.py#35\35]8;;\
                             version 0.12.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

d:\Project\Code\Diffusion_Data_Tools\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import subprocess

def wd14_tagging_image(working_folder):
    print("wd14 tagging")
    command = [
        r"D:\Project\Code\Kohya\kohya_ss\venv\Scripts\accelerate.EXE", "launch",
        #r"D:\Project\Code\Diffusion_Data_Tools\myenv\Scripts\accelerate.EXE", "launch",
        r"D:\Project\Code\Diffusion_Data_Tools\lib\tag_images_by_wd14_tagger.py",
        "--batch_size", "1",
        "--caption_extension", ".txt",
        "--caption_separator", ",",
        "--character_threshold", "0.5",
        "--max_data_loader_n_workers", "2",
        "--onnx",
        "--recursive",
        "--remove_underscore",
        "--repo_id", "SmilingWolf/wd-v1-4-convnextv2-tagger-v2",
        working_folder,
    ]

    subprocess.run(command)

def blip_tagging_image(working_folder):
    print("blip tagging")
    
    command = [
        #r'D:\Project\Code\Kohya\kohya_ss\venv\Scripts\python.exe',
        #r'D:/Project/Code/Kohya/kohya_ss/sd-scripts/finetune/make_captions.py',
        r"D:\Project\Code\Diffusion_Data_Tools\myenv\Scripts\python.exe",
        r'D:\Project\Code\Diffusion_Data_Tools\lib\make_captions.py',
        '--batch_size', '1',
        '--num_beams', '1',
        '--top_p', '0.9',
        '--max_length', '75',
        '--min_length', '5',
        '--beam_search',
        "--recursive",
        '--caption_extension', '.txt',
        working_folder,
        '--caption_weights', 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth'
    ]
    
    subprocess.run(command)

In [4]:
def rename_txt_files_old(directory, suffix='_wd14'):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt') and not file.endswith(f'{suffix}.txt'):
                old_file_path = os.path.join(root, file)
                new_file_path = os.path.join(root, f"{os.path.splitext(file)[0]}{suffix}.txt")
                if os.path.exists(new_file_path):
                    os.remove(new_file_path)
                os.rename(old_file_path, new_file_path)

def rename_txt_files(directory, suffix='_wd14'):
    for root, _, files in os.walk(directory):
        image_files = {os.path.splitext(file)[0] for file in files if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))}
        for file in files:
            if file.endswith('.txt') and not file.endswith(f'{suffix}.txt'):
                txt_base_name = os.path.splitext(file)[0]
                if txt_base_name in image_files:
                    old_file_path = os.path.join(root, file)
                    new_file_path = os.path.join(root, f"{txt_base_name}{suffix}.txt")
                    if os.path.exists(new_file_path):
                        os.remove(new_file_path)
                    os.rename(old_file_path, new_file_path)

def delete_flipped_images(directory, suffix='_wd14.txt'):
    # walk through the directory and its subfolders
    for root, dirs, files in os.walk(directory):
        flipped_images = [os.path.join(root, f) for f in files if f.endswith(suffix)]

        # delete the flipped images
        for flipped_image_path in flipped_images:
            os.remove(flipped_image_path)

In [5]:
def combine_and_cleanup_old(directory):
    for root, _, files in os.walk(directory):
        txt_files = [f for f in files if f.endswith('.txt') and not f.endswith('_wd14.txt')]
        for txt_file in txt_files:
            original_path = os.path.join(root, txt_file)
            wd14_path = original_path.replace('.txt', '_wd14.txt')

            if os.path.exists(wd14_path):
                with open(original_path, 'r', encoding='utf-8') as original_file:
                    original_content = original_file.read().replace('\n', ' ')

                with open(wd14_path, 'r', encoding='utf-8') as wd14_file:
                    wd14_content = wd14_file.read().replace('\n', ' ')

                combined_content = f"{original_content}, {wd14_content}"

                with open(original_path, 'w', encoding='utf-8') as combined_file:
                    combined_file.write(combined_content)

                os.remove(wd14_path)
                #print(f"Combined and deleted: {wd14_path}")



def combine_and_cleanup(directory):
    for root, _, files in os.walk(directory):
        txt_files = {f.replace('_wd14.txt', '').replace('_blip.txt', '') for f in files if f.endswith('_wd14.txt') or f.endswith('_blip.txt')}
        for base_name in txt_files:
            original_path = os.path.join(root, f"{base_name}.txt")
            wd14_path = os.path.join(root, f"{base_name}_wd14.txt")
            blip_path = os.path.join(root, f"{base_name}_blip.txt")

            contents = []

            if os.path.exists(blip_path):
                with open(blip_path, 'r', encoding='utf-8') as blip_file:
                    contents.append(blip_file.read().replace('\n', ' '))

            if os.path.exists(wd14_path):
                with open(wd14_path, 'r', encoding='utf-8') as wd14_file:
                    contents.append(wd14_file.read().replace('\n', ' '))

            combined_content = ', '.join(contents)
            
            if os.path.exists(original_path): os.remove(original_path)

            with open(original_path, 'w', encoding='utf-8') as combined_file:
                combined_file.write(combined_content)

            #if os.path.exists(wd14_path): os.remove(wd14_path)
            #if os.path.exists(blip_path): os.remove(blip_path)



#### main runner


In [7]:
working_folder = r'D:\Project\CivitAI\GameArt\hades\training data2' 
#delete_similar_image_in_subfolders(working_folder)

In [6]:
#delete_flipped_images(working_folder,'_wd14.txt')
wd14_tagging_image(working_folder)
rename_txt_files(working_folder,'_wd14')
blip_tagging_image(working_folder)
rename_txt_files(working_folder,'_blip')
combine_and_cleanup(working_folder)
#add_folder_name_to_files(working_folder, mode='all', tag_dropping_rate = 0.5, drop_chance = 0.75)
#rename_subfolders(working_folder)

wd14 tagging
blip tagging


In [11]:
combine_and_cleanup(working_folder)

In [6]:
working_folder = r'D:\Project\CivitAI\GameArt\hades\training data2'
combine_and_cleanup(working_folder)
add_folder_name_to_files(working_folder, mode='all_score', tag_dropping_rate = 0.5, drop_chance = 0.75)

In [16]:
working_folder = r"D:\Project\CivitAI\Spiderman\draft\any"

delete_flipped_images(working_folder,'_wd14.txt')
wd14_tagging_image(working_folder)
rename_txt_files(working_folder)
blip_tagging_image(working_folder)
combine_and_cleanup(working_folder)
add_folder_name_to_files(working_folder, mode='all', tag_dropping_rate = 0.5, drop_chance = 0.75)
#rename_subfolders(working_folder)

wd14 tagging


blip tagging


#### create prompt

In [1]:
import os, random

def get_text_files(directory):
    text_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                text_files.append(os.path.join(root, file))
    return text_files

def create_output_file(file_counter, out_path):
    output_file = os.path.join(out_path, f'test_prompt_{file_counter}.txt')
    open(output_file, 'w').close()
    return output_file

def remove_phrases(line, phrases=["from behind, ", "english text, ", "artist name, ", "twitter username, "]):
    for phrase in phrases:
        line = line.replace(phrase, "")
    return line


def process_file(txt_file, line_counter, file_counter, output_file, max_num_lines, suffix):
    with open(txt_file, 'r') as infile:
        for line in infile:
            line = remove_phrases(line)
            line_counter, file_counter, output_file = process_line(
                line, line_counter, file_counter, output_file, max_num_lines, suffix
            )
    return line_counter, file_counter, output_file

def process_subdir(sub_dir, line_counter, file_counter, output_file, max_num_lines, prompt_per_folder, suffix):
    text_files = get_text_files(sub_dir)
    if len(text_files) >= prompt_per_folder:
        selected_files = random.sample(text_files, prompt_per_folder)
        for txt_file in selected_files:
            line_counter, file_counter, output_file = process_file(
                txt_file, line_counter, file_counter, output_file, max_num_lines, suffix
            )
    return line_counter, file_counter, output_file

def process_directory(directory, max_num_lines, prompt_per_folder=2, out_path="", suffix=""):
    file_counter = 1
    line_counter = 0
    output_file = create_output_file(file_counter, out_path)

    for root, dirs, _ in os.walk(directory):
        for dir in dirs:
            sub_dir = os.path.join(root, dir)
            line_counter, file_counter, output_file = process_subdir(
                sub_dir, line_counter, 
                file_counter, output_file, 
                max_num_lines, prompt_per_folder, 
                suffix
            )

def process_line(line, line_counter, file_counter, output_file, max_num_lines, suffix):
    if line_counter >= max_num_lines:
        file_counter += 1
        line_counter = 0
        output_file = create_output_file(file_counter, os.path.dirname(output_file))
    with open(output_file, 'a') as outfile:
        prompt = (
            f'--prompt "score_9, score_8_up, score_7_up, volumetric lighting, '
            f'{line.strip()}, {suffix}" '
            '--negative_prompt "score_6_up, score_5_up, score_4_up, '
            'simple background, blurry, grayscale"\n'
        )
        outfile.write(prompt)
    return line_counter + 1, file_counter, output_file

In [2]:

working_folder = r'D:\Project\CivitAI\Spiderman\draft'
out_path = r'D:\Project\CivitAI\Spiderman\samping'
suffix  = "<lora:spiderverse_pony_v1-000075:1>"
max_num_lines = 12

os.makedirs(out_path, exist_ok=True)
process_directory(working_folder, max_num_lines, out_path = out_path, suffix=suffix)

## Other


In [5]:
import os
from PIL import Image

def  flip_img(directory):
    # specify the prefixes
    prefixes = ['f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16']

    # walk through the directory and its subfolders
    for root, dirs, files in os.walk(directory):
        for prefix in prefixes:
            # find all png images in the current directory that start with the current prefix
            images = [os.path.join(root, f) for f in files if f.startswith(prefix) and f.endswith('.png')]

            for image_path in images:
                # open the image
                img = Image.open(image_path)

                # horizontally flip the image
                img = img.transpose(Image.FLIP_LEFT_RIGHT)

                # save the flipped image, overwriting the original one
                img.save(image_path)

In [30]:
delete_flipped_images(r"D:\Project\CivitAI\Spiderman\draft\0other")

In [6]:
flip_img(r"D:\Project\CivitAI\kimetsu no yaiba\draft")

In [2]:
directory = r"D:\Project\CivitAI\GameArt\hades\training data2"
for root, _, files in os.walk(directory):
    print(files)

[]
['download (1).png', 'download (12).png', 'download (14).png', 'download (17).png', 'download (18).png', 'download (19).png', 'download (21).png', 'download (22).png', 'download (23).png', 'download (24).png', 'download (25).png', 'download (30).png', 'download (31).png', 'download (33).png', 'download (34).png', 'download (35).png', 'download (8).png', 'download (9).png', 'f_00001022.png', 'f_00001393.png', 'f_00001951.png', 'f_00002044.png', 'f_00002508.png', 'f_01000181.png', 'f_01001455.png', 'f_01001727_2.png', 'f_01001818.png', 'f_01004547.png', 'f_01004819.png', 'f_01004910.png', 'f_01005456.png', 'f_01005547.png']
[]
['1.png', '3431b99781c9a4bc34e33fd967241d05.png', 'ac06ed24d0978f76e43fdb5a5d1b8190.png', 'ae85b79408aa587a4de22e944ca7b46a.png', 'aphrodite-cosplay-by-justfaelia-tumblr-v0-v67ypwc5gf6a1.png', 'fa6088763f181bd6fe97549d65aeb001.png', 'r16a5clvj8p61.png', 'real (1).png', 'real (2).png', 'real (3).png', 'real (4).png', 'real (5).png', 'real (6).png', 'real (7).png'

In [26]:
import os
from PIL import Image

def delete_images(directory, target_number):
    # List all images and text files in the directory
    images = [f for f in os.listdir(directory) if f.endswith('.png')]
    texts = [f for f in os.listdir(directory) if f.endswith('.txt')]

    # If there are less or equal images than the target, do nothing
    if len(images) <= target_number:
        print("No need to delete images. Current number of images is less or equal to the target.")
        return

    # Dictionary to store image names and their minimum dimensions
    image_sizes = {}

    # Loop through the images and store their minimum dimensions
    for image in images:
        img_path = os.path.join(directory, image)
        with Image.open(img_path) as img:
            width, height = img.size
            min_size = min(width, height)
            image_sizes[image] = min_size

    # Sort the images by their minimum dimensions
    sorted_images = sorted(image_sizes.items(), key=lambda x: x[1])

    # Delete the smallest images until the target number is reached
    for i in range(len(sorted_images) - target_number):
        image_to_delete = sorted_images[i][0]
        img_path = os.path.join(directory, image_to_delete)
        os.remove(img_path)

    images = [f for f in os.listdir(directory) if f.endswith('.png')]
    # Delete text files that do not have a corresponding image
    for text in texts:
        img_name = f"{os.path.splitext(text)[0]}.png"
        if img_name not in images:
            txt_path = os.path.join(directory, text)
            os.remove(txt_path)

    print("Image resizing operation completed.")

def delete_txt(directory):
    # List all images and text files in the directory
    images = [f for f in os.listdir(directory) if f.endswith('.png')]
    texts = [f for f in os.listdir(directory) if f.endswith('.txt')]

    # Delete text files that do not have a corresponding image
    for text in texts:
        img_name = f"{os.path.splitext(text)[0]}.png"
        if img_name not in images:
            txt_path = os.path.join(directory, text)
            os.remove(txt_path)

In [23]:
from lib.library import delete_similar_images

#delete_similar_images(working_folder,similarity_threshold = 0.93)

In [27]:

working_folder = r"D:\Project\CivitAI\kimetsu no yaiba\draft\slayer\tanjiro kamado\2_bleading tanjiro"
delete_images(working_folder,100)

Image resizing operation completed.


In [25]:
delete_txt(working_folder)

In [1]:
import os

def rename_cropped_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.startswith("cropped_"):
                old_path = os.path.join(root, file)
                new_path = os.path.join(root, file[len("cropped_"):])
                os.rename(old_path, new_path)

# Replace 'cropped_' with the actual path to the folder
rename_cropped_images(r'D:\Project\CivitAI\Disney\princess comic\draft')
